In [ ]:
import pandas as pd
import plotly.express as px
from datetime import datetime, time
import pytz
import random

# ---------------- LOAD DATA ----------------
df = pd.read_csv("Play Store Data.csv")

# ---------------- CLEAN INSTALLS ----------------
df['Installs'] = df['Installs'].astype(str)\
    .str.replace(',', '', regex=False)\
    .str.replace('+', '', regex=False)

df = df[df['Installs'].str.isnumeric()]
df['Installs'] = df['Installs'].astype(int)

# ---------------- TOTAL INSTALLS BY CATEGORY ----------------
category_installs = df.groupby('Category', as_index=False)['Installs'].sum()

# ---------------- REMOVE A, C, G, S ----------------
category_installs = category_installs[
    ~category_installs['Category'].str.startswith(('A','C','G','S'))
]

# ---------------- TOP 5 ----------------
top5 = category_installs.nlargest(5, 'Installs')['Category']
df_top5 = df[df['Category'].isin(top5)].copy()

# ---------------- ADD DEMO COUNTRIES ----------------
countries = ['India','USA','UK','Canada','Germany','France','Brazil','Australia']
df_top5['Country'] = [random.choice(countries) for _ in range(len(df_top5))]

# ---------------- HIGHLIGHT ----------------
df_top5['Highlight'] = df_top5['Installs'].apply(
    lambda x: 'Above 1M' if x > 1_000_000 else 'Below 1M'
)

# ---------------- CREATE CHOROPLETH DATA ----------------
choropleth_df = (
    df_top5
    .groupby(['Country','Category','Highlight'], as_index=False)['Installs']
    .sum()
)

# ---------------- TIME CHECK (6 PM – 8 PM IST) ---------------- 18 20 
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist).time()

start_time = time(18, 0)   # 6 PM 18
end_time   = time(20, 0)   # 8 PM

# ---------------- SHOW MAP ----------------
if start_time <= now <= end_time:
    fig = px.choropleth(
        choropleth_df,
        locations="Country",
        locationmode="country names",
        color="Installs",
        hover_name="Category",
        hover_data=["Highlight"],
        animation_frame="Category",
        color_continuous_scale="Viridis",
        title="Global Installs by Top 5 App Categories"
    )
    fig.show()
else:
    print("⏰ Choropleth visible only between 6 PM and 8 PM IST")


C:\Users\sowmi\AppData\Local\Temp\ipykernel_8432\820661958.py:55: DeprecationWarning: The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.
  fig = px.choropleth(
